In [62]:
import numpy as np
import os
from matplotlib import pyplot as plt
# plt.rcParams['image.cmap'] = 'gray'
os.environ["KERAS_BACKEND"] = "torch"
import keras_core as keras

In [63]:
model = keras.models.Sequential([
    keras.layers.Input((3,)),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(3, activation="relu"),
])

In [64]:
import torch

x_train = np.random.rand(64, 3)
y_train = np.zeros((64, 3))

train_dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(x_train), torch.from_numpy(y_train)
)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=16, shuffle=True
)



In [65]:
optimizer = keras.optimizers.Adam(learning_rate=1e-2)

def my_loss(target, output):
    return keras.ops.abs(42 - keras.ops.mean(output))

loss_fn = my_loss

In [66]:
model.layers[1].trainable = False
model.compile()

In [67]:
epochs = 100
for epoch in range(epochs):
    print("epoch", epoch)
    for step, (inputs, targets) in enumerate(train_dataloader):
        # for frozen_layer_ix in range(2):
        # Forward pass
        weights_before = model.layers[1].get_weights()
        output = model(inputs)
        loss = loss_fn(targets, output)

        # Backward pass
        model.zero_grad()
        trainable_weights = [v for v in model.trainable_weights]

        # Call torch.Tensor.backward() on the loss to compute gradients
        # for the weights.
        loss.backward()
        gradients = [v.value.grad for v in trainable_weights]

        # Update weights
        with torch.no_grad():
            optimizer.apply(gradients, trainable_weights)

        if False not in [(weights_before[i] == model.layers[1].get_weights()[i]).all() for i in range(len(weights_before))]:
            print('the frozen weights did not change')
        else:
            raise Exception("the weights of the frozen layer changed")

        print(
            f"Training loss (for 1 batch) at step {step}: {loss.detach().numpy():.4f}"
        )

epoch 0
the frozen weights did not change
Training loss (for 1 batch) at step 0: 41.8906
the frozen weights did not change
Training loss (for 1 batch) at step 1: 41.8863
the frozen weights did not change
Training loss (for 1 batch) at step 2: 41.8412
the frozen weights did not change
Training loss (for 1 batch) at step 3: 41.8639
epoch 1
the frozen weights did not change
Training loss (for 1 batch) at step 0: 41.8165
the frozen weights did not change
Training loss (for 1 batch) at step 1: 41.7796
the frozen weights did not change
Training loss (for 1 batch) at step 2: 41.7865
the frozen weights did not change
Training loss (for 1 batch) at step 3: 41.8073
epoch 2
the frozen weights did not change
Training loss (for 1 batch) at step 0: 41.7425
the frozen weights did not change
Training loss (for 1 batch) at step 1: 41.7190
the frozen weights did not change
Training loss (for 1 batch) at step 2: 41.7050
the frozen weights did not change
Training loss (for 1 batch) at step 3: 41.6457
epoc